In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
from sklearn.model_selection import train_test_split 
from matplotlib import pyplot as plt
%matplotlib inline 

In [ ]:
!unzip /kaggle/input/facial-keypoints-detection/training.zip
!unzip /kaggle/input/facial-keypoints-detection/test.zip

In [ ]:
IdLookupTable = pd.read_csv('/kaggle/input/facial-keypoints-detection/IdLookupTable.csv')
IdLookupTable.info()

In [ ]:
IdLookupTable.head()

In [ ]:
SampleSubmission = pd.read_csv('/kaggle/input/facial-keypoints-detection/SampleSubmission.csv')
SampleSubmission.info()

In [ ]:
SampleSubmission.head()

In [ ]:
test = pd.read_csv('test.csv')
test.info()

In [ ]:
test.head()

In [ ]:
training = pd.read_csv('training.csv')
training.info()

In [ ]:
training.head(2)

In [ ]:
training = training.dropna()
#training.fillna(method = 'ffill',inplace = True)

In [ ]:
training.shape, type(training)

In [ ]:
training['Image'] = training['Image'].apply(lambda x: np.fromstring(x, dtype=int, sep=' ').reshape((96,96)))

In [ ]:
test['Image'] = test['Image'].apply(lambda x: np.fromstring(x, dtype=int, sep=' ').reshape((96,96)))

In [ ]:
def get_image_and_dots(df, index):
    image = plt.imshow(df['Image'][index],cmap='gray')
    l = []
    for i in range(1,31,2):
        l.append(plt.plot(df.loc[index][i-1], df.loc[index][i], 'ro'))
        
    return image, l


In [ ]:
fig = plt.figure(figsize=(8, 8))
fig.subplots_adjust(
    left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)

for i in range(16):
    ax = fig.add_subplot(4, 4, i + 1, xticks=[], yticks=[])
    get_image_and_dots(training, i)

plt.show()

In [ ]:
X = np.asarray([training['Image']], dtype=np.uint8).reshape(training.shape[0],96,96,1)
y = training.drop(['Image'], axis=1).to_numpy()

In [ ]:
y1 = training.drop(['Image'], axis=1)

In [ ]:
type(X), type(y)

In [ ]:
from tensorflow.python.keras.utils.data_utils import Sequence
from keras import layers , models
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
import albumentations as A

In [ ]:
class DataLoader(Sequence):
    def __init__(self, X, y, batch_size, augmentations=None, as_rgb=False):
        self.X, self.y = X, y
        self.batch_size = batch_size
        self.augment = augmentations
        self.shuffle = True
        self.as_rgb = as_rgb
        self.on_epoch_end()

    def __len__(self):
        """ Corresponds to the number of steps in one epoch. """
        return int(np.ceil(len(self.X) / float(self.batch_size)))

    def __getitem__(self, idx):
        indexes = self.indexes[idx*self.batch_size: (idx+1)*self.batch_size]
        batch_X = self.X[indexes, ...]
        batch_y = self.y[indexes, :]
        
        # Convert grayscale to RGB if needed (if you want to use a pre-trained ResNet for example)
        if self.as_rgb:
            batch_X = np.tile(batch_X, reps=(1,1,1,3))

        # Apply transformations on both images and keypoints
        if self.augment is not None:
            keypoints = np.array([ tuple(zip(point[::2], point[1::2])) for point in batch_y ])
            transformed = [ self.augment(image=x, keypoints=y) for x,y in zip(batch_X, keypoints) ]
            batch_X = np.stack([ z['image'] for z in transformed ], axis=0)
            batch_y = np.stack([ np.array(z['keypoints']).flatten(order='C') for z in transformed ], axis=0)

        return batch_X, batch_y

    def on_epoch_end(self):
        """ Shuffle the data after each epoch to avoid oscillation patterns in the loss. """
        self.indexes = np.arange(len(self.X))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train2, X_valid, y_train2, y_valid = train_test_split(X_train, y_train, test_size=0.10, shuffle=True)

# Define augmentation strategy
transform = A.Compose([
    A.ShiftScaleRotate(rotate_limit=30, p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.RandomCrop(width=96, height=96, p=1),
    A.Rotate(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.GaussianBlur(p=0.5),
    A.GaussNoise(var_limit=(1e-5, 1e-3), p=0.5),
], keypoint_params=A.KeypointParams(format='xy', remove_invisible=False))

train_loader = DataLoader(X_train2, y_train2, batch_size=128, augmentations=transform)
print(X_train2.shape, y_train2.shape)
print(X_valid.shape, y_valid.shape)

# Visualize augmented data
x_batch, y_batch = train_loader[1]
show_random_samples(x_batch.squeeze(), y_batch)

Model

In [ ]:
model = models.Sequential()

model.add(layers.Convolution2D(32 , (3,3) , input_shape = (96 , 96 , 1) ))
model.add(layers.ReLU())
model.add(layers.BatchNormalization())
          

model.add(layers.Convolution2D(64 , (3,3) ))
model.add(layers.ReLU())
model.add(layers.MaxPool2D())
          
model.add(layers.Convolution2D(128 , (3,3) ))
model.add(layers.ReLU())
model.add(layers.MaxPool2D())
          
model.add(layers.Convolution2D(256 , (3,3) ))
model.add(layers.ReLU())
model.add(layers.MaxPool2D())
          
model.add(layers.Convolution2D(512 , (3,3) ))
model.add(layers.ReLU())
model.add(layers.MaxPool2D())
          
model.add(layers.Convolution2D(1024 , (3,3) ))
model.add(layers.ReLU())
model.add(layers.MaxPool2D())
          
          
model.add(layers.Flatten())
model.add(layers.Dense(512 , activation = 'relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(30))#Any customization on the training logic is then easily controlled by the user.


          
model.summary()

In [ ]:
model.compile(optimizer = 'adam' , loss = 'mean_squared_error' , metrics = ['mae'])

In [ ]:
history = model.fit(X_train2, y_train2 , epochs = 500 , batch_size = 32 , validation_data = (X_valid, y_valid))


In [ ]:
model.save('keypoints.h5')

In [ ]:
test.shape, type(test)

In [ ]:
test_X = np.asarray([test['Image']], dtype=np.uint8).reshape(test.shape[0],96,96,1)
test_res = model.predict(test_X)

In [ ]:
test_res = model.predict(test_X)

In [ ]:
train_predicts = model.predict(X_train)

In [ ]:
n = 11

xv = X_test[n].reshape((96,96))
plt.imshow(xv,cmap='gray')

for i in range(1,31,2):
    plt.plot(test_res[n][i-1], test_res[n][i], 'ro')
    plt.plot(y_test[n][i-1], y_test[n][i], 'x', color='green')

plt.show()

In [ ]:
header = list(y1.columns)


In [ ]:
test_predicts = pd.DataFrame(test_res, columns = header)

In [ ]:
for i in range(IdLookupTable.shape[0]):
    IdLookupTable.Location[i] = test_predicts.loc[IdLookupTable.ImageId[i]-1][IdLookupTable.FeatureName[i]]
    

In [ ]:
SampleSubmission.Location = IdLookupTable.Location

In [ ]:
my_submission = SampleSubmission

In [ ]:
my_submission.to_csv('submission1.csv', index=False)

In [ ]:
my_submission

In [ ]:
samplesub = pd.read_csv('../input/facial-keypoints-detection/SampleSubmission.csv')
samplesub

In [ ]:
idlookup_data = IdLookupTable

In [ ]:
feature_names = list(idlookup_data['FeatureName'])
image_ids = list(idlookup_data['ImageId']-1)
row_ids = list(idlookup_data['RowId'])

feature_list = []
for feature in feature_names:
    feature_list.append(feature_names.index(feature))
    
predictions = []
for x,y in zip(image_ids, feature_list):
    predictions.append(test_res[x][y])
    
row_ids = pd.Series(row_ids, name = 'RowId')
locations = pd.Series(predictions, name = 'Location')
locations = locations.clip(0.0,96.0)
submission_result = pd.concat([row_ids,locations],axis = 1)
submission_result.to_csv('submission.csv',index = False)